# **Overlapping Community Detection in Protein-protein interaction network with GCN**





---
*Classification mode has been changed to multi label classification*



Upload the files PPI-Net.txt and Original-Communities.txt

In [ ]:
%tensorflow_version 1.x

**Total Number of unique nodes in node file**

In [ ]:
L =[]
f = open("/content/PPI-Net.txt","r")
for line in f:
  for i in line.split()[0:2]:
    if i not in L:
      L.append(i)
f.close()
print(len(L))

1624


**Total Number of unique nodes in community file**

In [ ]:
P=[]
f = open("/content/Original-Communities.txt","r")
for line in f:
  for i in line.split():
        if i in line.split() and i not in P:
          P.append(i)
f.close()
print(len(P))

1189


**Nodes which are in community file but not in node file**

In [ ]:
c = 0
for i in P:
  if i not in L:
    c = c + 1 
print(c)

416


**Nodes which are in node file but not in community file**

In [ ]:
c = 0
for i in L:
  if i not in P:
    c = c + 1 
print(c)

851


**Common nodes to both the files**

In [ ]:
c = 0
for i in L:
  if i in P:
    c = c + 1 
print(c)

773


**Import all the necessary libraries, paths, and constants**

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
import math
import random
#define path of graph file and communities file
labelfile = '/content/Original-Communities.txt'
graphfile =  '/content/PPI-Net.txt'
#select number of communities with maximum members
n = 100 #min = 1, max = 203

**Function for loading data from graph file and ground truth file**

In [ ]:
def loadData(labelfile, graphfile):
  data = []
  n_communities = []
  #save all communities in list-data regardless of the order
  with open(labelfile) as inputfile:
      for line in inputfile:
          data.append(line.split())
  graph = {}
  #save n communities with maximum members in list-n_communities keeping the decreasing order of length
  # print(data)
  for i in range(n):
    # print(i)
       p = max(data,key=len)
       n_communities.append(p)
       data.remove(p)
  with open(graphfile) as inputfile:
        for line in inputfile:
            node = line.split()[0]#read 1st element in any column(node 1)
            neigh = line.split()[1]#read 2nd element in any column(node 2)
            #Not include those nodes, which are not in n_communities
            g = 0
            for i in range(n):
              #  print(i)
               if( node in n_communities[i]):
                     g=1
            if g==0:
              continue
            g1=0
            for i in range(len(n_communities)):
               if( neigh in n_communities[i]):
                     g1=1
            if g1==0:
              continue
            #save all the nodes and edges, which are common to the graphfile and n_communities
            if node in graph:
                graph[node].add(neigh)
            else:
                graph[node] = {neigh}
            if neigh in graph:
                graph[neigh].add(node)
            else:
                graph[neigh] = {node}
            # print(line)
  return n_communities, graph

*The below function will output a multi label-Binarizer label for every node. It takes a list of all the communities having a particular node as the member. If n = 5, the length of this multi label-Binarizer will be 5, which covers the possibility of nodes belonging to atleast one community out of 5 by putting 1 at the respective place of presence. [10101] tells that node is present in community1, community3, and community5.*



In [ ]:
def labeller(pos):
  p = [0]*n
  for i in pos:
     p[i]=1
  return  p

**Preparing adjacency matrix, degree Matrix, and labels of nodes**

In [ ]:
def preprocessingData(graph, n_communities):
  adj_matrix = np.zeros((len(graph), len(graph)))
  degree_matrix = np.zeros((len(graph), len(graph)))
  node_index = {}
  j = 0
  for i in graph:
    node_index[i] = j
    j += 1
  for i in node_index:
    for j in graph[i]:
      if(j in node_index):
        adj_matrix[node_index[i]][node_index[j]] = 1
    degree_matrix[node_index[i]][node_index[i]] = len(graph[i])
  label = np.zeros((len(graph), n))
  for nodeId in node_index:
    u=[]
    for i in n_communities:
      if nodeId in i:
        u.append(n_communities.index(i))
    label[node_index[nodeId]]=labeller(u)
  return adj_matrix, degree_matrix, label,node_index

**Building GNN, training, and testing it**

---
For multi-label classification problem, tf.nn.softmax_cross_entropy_with_logits() is being replaced by tf.nn.sigmoid_cross_entropy_with_logits() and tf.nn.softmax() with tf.nn.sigmoid().


In [ ]:
def gnn(adj_matrix, degree_matrix,feature_matrix,label,train_index):
    features = tf.placeholder(tf.float32, shape = ((None,len(feature_matrix))))
    adjacency = tf.placeholder(tf.float32, shape = ((None,None)))
    degree = tf.placeholder(tf.float32, shape = ((None,None)))
    labels = tf.placeholder(tf.float32, shape = ((None,n)))
    weights1 = tf.Variable(tf.random_normal([len(feature_matrix),512], stddev = 1))
    weights2 = tf.Variable(tf.random_normal([512,n], stddev = 1))
    trainIndex = tf.placeholder(tf.int32, shape = ((len(train_index))))
#Defining GCN layer
    def layer(features, adjacency, degree, weights):
        with tf.name_scope('gcn_layer'):
            d_ = tf.pow(tf.matrix_inverse(degree), 0.5)
            y = tf.matmul(d_, tf.matmul(adjacency, d_))
            kernel = tf.matmul(features, weights)
            return tf.nn.relu(tf.matmul(y, kernel))

#Building with GCN layer        
    hidden1 = layer(features, adjacency, degree, weights1)
    hidden1 = tf.layers.dropout(hidden1, rate=0.5)
    model = layer(hidden1, adjacency, degree, weights2)
    training_output = tf.gather(model, trainIndex)

#Defining loss function and optimizer
    cross_entropy    = tf.nn.sigmoid_cross_entropy_with_logits(logits = training_output,labels = tf.cast(label,tf.float32))
    loss             = tf.reduce_mean(tf.reduce_sum(cross_entropy, axis=1))
    train_op        = tf.train.AdamOptimizer().minimize(loss)
#creating session
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)

#training up to 2000 epochs
    for i in range(2000):
        _, cost = sess.run([train_op, loss], feed_dict = {features: feature_matrix, adjacency: adj_matrix, degree: degree_matrix, labels: label, trainIndex: train_index})
        if(i%1 == 0):
            predict = sess.run(tf.cast(tf.sigmoid(model) > 0.5, tf.int32,name='predictions'), feed_dict = {features: feature_matrix, adjacency: adj_matrix, degree: degree_matrix,labels: label})
        print("epoch:",i) 
    return predict,label, training_output


if __name__ == '__main__':
  n_communities,graph = loadData(labelfile, graphfile)
  adj_matrix, degree_matrix, label,nd = preprocessingData(graph, n_communities)
#calculating feature matrix from adj_matrix
  feature_matrix = np.eye(len(adj_matrix))
  adj_matrix = adj_matrix + feature_matrix
  train_index = []
  for i in range(len(adj_matrix)):
      train_index.append(i)
  test_res,test_label,t= gnn(adj_matrix, degree_matrix, feature_matrix, label,train_index)

/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10
epoch: 11
epoch: 12
epoch: 13
epoch: 14
epoch: 15
epoch: 16
epoch: 17
epoch: 18
epoch: 19
epoch: 20
epoch: 21
epoch: 22
epoch: 23
epoch: 24
epoch: 25
epoch: 26
epoch: 27
epoch: 28
epoch: 29
epoch: 30
epoch: 31
epoch: 32
epoch: 33
epoch: 34
epoch: 35
epoch: 36
epoch: 37
epoch: 38
epoch: 39
epoch: 40
epoch: 41
epoch: 42
epoch: 43
epoch: 44
epoch: 45
epoch: 46
epoch: 47
epoch: 48
epoch: 49
epoch: 50
epoch: 51
epoch: 52
epoch: 53
epoch: 54
epoch: 55
epoch: 56
epoch: 57
epoch: 58
epoch: 59
epoch: 60
epoch: 61
epoch: 62
epoch: 63
epoch: 64
epoch: 65
epoch: 66
epoch: 67
epoch: 68
epoch: 69
epoch: 70
epoch: 71
epoch: 72
epoch: 73
epoch: 74
epoch: 75
epoch: 76
epoch: 77
epoch: 78
epoch: 79
epoch: 80
epoch: 81
epoch: 82
epoch: 83
epoch: 84
epoch: 85
epoch: 86
epoch: 87
epoch: 88
epoch: 89
epoch: 90
epoch: 91
epoch: 92
epoch: 93
epoch: 94
epoch: 95
epoch: 96
epoch: 97
epoch: 98
epoch: 99
epoch: 100

***Writing the result in the file***

In [ ]:
key_list = list(nd.keys())
val_list = list(nd.values())
G={}
for i in range(len(key_list)):
  position = val_list.index(i)
  G[key_list[position]] = [j for j,x in enumerate(list(test_res[i])) if x == 1]
final_L = []
for i in range(n):
  final_L.append([])
for i in G:
  for j in G[i]:
    final_L[j].append(i)
with open(str(n) +'_original_communities_result.txt', 'w') as file:
    for item in final_L:
      for o in item:
            file.write("%s" % o + "\t")
      file.write("\n")

***Total nodes participating in classification***

In [ ]:
len(key_list)

682

# ***Evaluation Metrics***

In [ ]:
import numpy as np
import sklearn

if __name__ == "__main__":
    print('Subset accuracy: {0}'.format(sklearn.metrics.accuracy_score(test_label, test_res, normalize=True, sample_weight=None)))
    print('Hamming loss: {0}'.format(sklearn.metrics.hamming_loss(test_label, test_res)) )